# Stall prediction

 Prediction of the pressure profil of an aircraft wing based on deep learning.
 Our goal is to use pressure predictions to prevent aircraft stalls.
 The advantage of deep learning in this case is that it requires very little computation compared with real-time simulations, and delivers rapid results that can be used by pilots or drones in their piloting

We are using data coming from the deepstall project : https://projects.asl.ethz.ch/datasets/doku.php?id=deepstall

The data we have is in h5df, to visualize it, we can use : https://myhdf5.hdfgroup.org/

Important question : Continuous regression or classification ?

Goal : predict pressure field (dp1, dp2, dp3, dp4)

 ## Data Importation

We have 2 dataset : one in wind tunnel and another in real flight condition

In [109]:
import pandas as pd # Data Managing
import seaborn as sns # visual tool
import matplotlib.pyplot as plt # visual tool
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import random


import h5py # to deal with h5p file



file_FD = h5py.File('Flight_Data/FD_cal.h5', "r")
file_WT = h5py.File('Wind_Tunnel/WT_cal.h5', "r")

FD_keys = list(file_FD.keys())
WT_keys = list(file_WT.keys())

FD_attribute =list(file_FD[FD_keys[0]].keys())
print(len(FD_attribute)) #548

WT_attribute =list(file_WT[WT_keys[0]].keys())
print(len(WT_attribute)) #506 

data_WT = {}
data_FD = {}
list_keys_WT = list(file_WT.keys())
list_keys_FD = list(file_FD.keys())
print(list_keys_FD)


548
506
['07_54_01.ulg', '08_12_24.ulg', '08_29_12.ulg', '08_39_56.ulg', '08_47_27.ulg', '08_50_41.ulg', '09_14_44.ulg', '09_14_52.ulg', '09_40_38.ulg', '09_58_51.ulg', '10_19_49.ulg', '10_53_14.ulg', '11_12_32.ulg', '13_00_54.ulg', '13_14_24.ulg', '13_53_32.ulg', '14_10_28.ulg', '14_29_32.ulg']


## Surface cleaning

In [33]:

        
common_attr = ['airflow_aoa_0.aoa_rad', 'airflow_slip_0.slip_rad', 'airspeed_0.true_airspeed_m_s', 'airspeed_1.true_airspeed_m_s',
     'sdp31module_0.dP1_4', 'sdp31module_0.dP1_3', 'sdp31module_0.dP1_2','sdp31module_0.dP1_1']


In [34]:
print("cleaned data column number : " + str(len(common_attr)) + " uncleaned :"  + str(len(FD_attribute)))

cleaned data column number : 8 uncleaned :548


### From h5d to dict and to pandas dataframe format

In [35]:

# for each key of each h5d file we create a dict which will be later transform into a pandas dataframe
dicts_list_WT = [{} for _ in range(len(list_keys_WT))]
dicts_list_FD = [{} for _ in range(len(list_keys_FD))]

# Populate the list of dict
for _ in range(len(list_keys_WT)):
    for attr in common_attr:
        dicts_list_WT[_][attr] = file_WT[list_keys_WT[_]][attr][0]
for _ in range(len(list_keys_FD)):
    for attr in common_attr:
        dicts_list_FD[_][attr] = file_FD[list_keys_FD[_]][attr][0]
    
global_dict_list = dicts_list_FD + dicts_list_WT


### From dicts to pandas data frame

an important decision is to be made here: are we going to mix all our data in one massive dataframe ? Or are we going to make small ones ?

For each keys (for exemple '11_aoa_sweep_m5') of the dataframe there is an environmental context, for exemple the experiment as realized in a wind tunnel or outside, during a foggy day or not, using this attack angle or this one. All those informations have an impact on the data, so if we mix datas we also mix contexts and we lose the **contextual information**.



In [70]:


df_list_WT = [pd.DataFrame() for _ in range(3)]
for _ in range(len(df_list_WT)):
    df_list_WT[_] = pd.DataFrame.from_dict(dicts_list_WT[_])
    #rm_constant_field(df_list[_]) might not be not a good idea

#df_list_FD = [pd.DataFrame() for _ in range(len(dicts_list_FD))]
#for _ in range(len(df_list)):
    #df_list[_] = pd.DataFrame.from_dict(dicts_list_FD[_])
    #rm_constant_field(df_list[_]) might not be not a good idea    


DF = pd.concat(df_list_WT)
print(DF.shape)


(15727, 8)


TODO:
1) Reduce the dataset to core information
2) Prepare the data for the MLP:  data preparaion: data normalization, 
3) Create MLP and set hyperparameter: Net class creation : Layers, numb of layers, activaion func, (with torch)
4) Define training set, epochs, 

## Data preparation

In [76]:
scaler = StandardScaler()
DF_numpy = DF.to_numpy()
DF_scaled = scaler.fit_transform(DF_numpy)
DF_scaled = pd.DataFrame(DF_scaled, columns = common_attr)
DF_scaled.head()

,airflow_aoa_0.aoa_rad,airflow_slip_0.slip_rad,airspeed_0.true_airspeed_m_s,airspeed_1.true_airspeed_m_s,sdp31module_0.dP1_4,sdp31module_0.dP1_3,sdp31module_0.dP1_2,sdp31module_0.dP1_1
0,-0.557741,-0.786432,-0.676724,-0.232048,0.287452,0.529745,0.325550,0.725582
1,-0.553250,-0.678135,-0.665277,-0.228996,0.284494,0.525687,0.329221,0.725168
2,-0.543928,-0.602315,-0.653831,-0.225943,0.284494,0.527774,0.357780,0.747474
3,-0.533110,-0.610619,-0.648216,-0.223654,0.289548,0.530903,0.345967,0.746003
4,-0.522301,-0.618923,-0.647792,-0.222054,0.290042,0.528041,0.323777,0.737320


Define train data and test data

In [104]:
print(DF_scaled.shape)


X = DF.drop(['sdp31module_0.dP1_4', 'sdp31module_0.dP1_3', 'sdp31module_0.dP1_2','sdp31module_0.dP1_1'], axis = 1)
Y = DF.drop(['airflow_aoa_0.aoa_rad', 'airflow_slip_0.slip_rad', 'airspeed_0.true_airspeed_m_s', 'airspeed_1.true_airspeed_m_s'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

X_train.head()

(15727, 8)


,airflow_aoa_0.aoa_rad,airflow_slip_0.slip_rad,airspeed_0.true_airspeed_m_s,airspeed_1.true_airspeed_m_s
8338,-0.001504,-0.004384,10.561196,9.693210
7160,-0.013948,0.001444,10.508673,9.659739
7843,-0.001934,-0.000443,10.546426,9.683150
2600,0.159616,-0.010455,11.154937,10.898161
747,0.166515,0.035636,11.149666,10.838683


# Model Design

## First results

MLP

In [107]:
nn = MLPRegressor(
    activation='relu',
    hidden_layer_sizes=(4, 10, 40, 10, 4),
    alpha=0.001,
    random_state=20,
    early_stopping=False)

nn.fit(X_train,y_train)


#grid search


C:\Users\alexe\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(alpha=0.001, hidden_layer_sizes=(4, 10, 40, 10, 4),
             random_state=20)

In [108]:
pred = nn.predict(X_test)

print('score :', nn.score(X_test, y_test))

score : 0.387344290806361


## Hyperparameters Tuning

The hyperparameters of our model are common to all MLP models: 
 - Learning rate
 - numbers of layers
 - number of node per layer
 - (random state fixed to 20)

We will search for optimal with Randomized Parameter Optimization (RandomizedSearchCV)

In [117]:
from sklearn.model_selection import RandomizedSearchCV

input_size = 4
output_size = 4

max_hidden_layer = 5
min_hidden_layer = 1

nb_layer = int(np.floor(random.uniform(min_hidden_layer, max_hidden_layer + 1)))
print(nb_layer)

max_node_number = 30
min_node_number = 10


hidden_layer_sizes = [ int(random.uniform(min_node_number, max_node_number + 1)) for _ in range(nb_layer)]
print(hidden_layer_sizes)



2
[21, 15]


In [181]:
def ar_double_array(list1, list2):
    pass_ = False
    index_  = 0
    q = 0
    j = 0
    for i in range(len(list1)):
        index_ = i
        j = index_
        while j <= len(list1) - 1:
            if(list1[index_] < list1[j]):
                    max_ = list1[j]
                    q = list1[index_]
                    list1[index_] = list1[j]
                    list1[j] = q
                    q = list2[index_]
                    list2[index_] = list2[j]
                    list2[j] = q
                    j = index_
            j += 1
    #print(list1)
            
    return list2



#print(ar_double_array([1, 2, 3, 4, 5], [21, 32, 76, 11, 101]))

learning_rate = 0.005
random_state = 20

def gen_pop(population_size, max_hidden_layer, min_hidden_layer, max_node_number, min_node_number, input_size, output_size):
    hidden_layer_sizes = []
    population = []
    for _ in range(population_size):
        nb_layer = int(np.floor(random.uniform(min_hidden_layer, max_hidden_layer + 1)))
        hidden_layer_sizes = [ int(random.uniform(min_node_number, max_node_number + 1)) for _ in range(nb_layer)]
        hidden_layer_sizes.append(output_size)
        hidden_layer_sizes.insert(0, input_size)
        if hidden_layer_sizes not in population:
            population.append(hidden_layer_sizes)
    return population

def fitness_func(genotype, X_train, y_train,  X_test, y_test):
    nn = MLPRegressor(
    activation='relu',
    hidden_layer_sizes=genotype,
    alpha=learning_rate,
    random_state=20,
    early_stopping=False)

    nn.fit(X_train,y_train)

    return nn.score(X_test, y_test)

def fitness_func_test(genotype):
    som = 0
    for element in genotype:
        if(element < 17):
            som+=1
    return som
          

def fitness_func_class(population):
    scores = []
    print(len(population))
    for _ in range(len(population)):
        scores.insert(_, fitness_func_test(population[_]))
    print(scores)
    return ar_double_array(scores, population)
    

def xover_population(population):
    nb_kids = int(len(population) / 3)
    index = 0
    new_pop = []
    kid = []

    for _ in range(nb_kids):
        cut = int(min(len(population[index]), len(population[index + 1]))/2)
        kid += population[index][:cut]
        kid += population[index + 1][cut:]
        index += 1
        new_pop.append(kid)
        kid = []
    return new_pop

def mutation(population, odd1, odd2):
    for _ in range(len(population)):
        if random.uniform(1, 100) < odd1:
            for i in range(len(population[_])):
                if random.uniform(1, 100) < odd2:
                    if random.uniform(1, 100) < 50:
                        population[_][i] += 6
                    else:
                        population[_][i] -= 6
    return population
def gen_alg_data(population_size,max_hidden_layer, min_hidden_layer, max_node_number, min_node_number, input_size, output_size,
                 X_train, y_train, X_test, y_test):
    population = gen_pop(population_size, max_hidden_layer, min_hidden_layer, max_node_number, min_node_number, input_size, output_size)

    while(len(population) > 2):
        population = fitness_func_class(population)
        population = xover_population(population)
        population = mutation(population)
        
    return population

pop = [ [ int(random.uniform(min_node_number, max_node_number + 1)) for _ in range(int(random.uniform(min_hidden_layer, max_hidden_layer)))] for _ in range(60)]
print(pop)

pop = fitness_func_class(pop)
print(pop)

pop = xover_population(pop)
print(pop)

pop = fitness_func_class(pop)

pop = xover_population(pop)
print(pop)

pop = fitness_func_class(pop)

pop = xover_population(pop)
print(pop)

pop = fitness_func_class(pop)
print(pop)

pop = xover_population(pop)
print(pop)


[[24, 19, 15], [19, 16, 12], [21, 28, 27, 20], [10, 27], [14, 23], [27, 16], [15, 26, 19], [18, 21, 18, 15], [18], [26, 13, 14, 25], [21, 29, 29, 28], [10, 20, 19, 18], [23, 21, 26, 15], [19, 19, 15, 13], [26, 26, 15], [23, 25], [29, 12, 11], [10, 19, 14], [13, 25], [14], [22, 18, 25, 16], [13, 11], [13, 13, 22, 25], [27, 17], [24, 13], [13, 12], [10, 10, 27, 10], [11, 17, 11, 19], [13, 22], [16, 27], [30, 25, 14, 27], [29, 16], [19, 13], [29], [22, 11, 30], [10], [27], [30, 17, 25, 19], [24, 18], [22, 17, 11, 24], [24, 23, 14], [27, 26, 26], [27, 27, 16, 25], [11, 26, 17, 27], [22], [17, 11, 30], [14], [11, 14], [16], [10, 24], [22, 30], [24, 26, 20], [19], [23, 26], [11, 10, 24], [13], [16], [12, 30, 30, 22], [12, 28], [24, 17]]
60
[1, 2, 0, 1, 1, 1, 1, 1, 0, 2, 0, 1, 1, 2, 1, 0, 2, 2, 1, 1, 1, 2, 2, 0, 1, 2, 3, 2, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0]
[[10, 10, 27, 10], [26, 13, 14, 25], [19, 19, 15, 13], [29, 12, 11], [10, 1

## Draw

# Dump

In [ ]:
# Then we transform each dict into a pandas dataframe and check if there is not any constant field
def rm_constant_field(df):
    """
    Drops constant value columns of pandas dataframe.
    """
    result = df.copy()
    for column in df.columns:
        if len(df[column].unique()) == 1:
            result = result.drop(column,axis=1)
    return result


def id_common_constant(df_list):
    list_set_each = [set() for _ in range(len(df_list))]
    for _ in range(len(df_list)):
        for column in df_list[_].columns:
            if(df_list[_][column].unique()) == 1:
                list_set_each[_].add(column)
    common_set = set.intersection(*list_set_each)
    return common_set
